# Section 1-3 - Parameter Tuning

In previous sections, we took the approach of using Scikit-learn as a black box. We now review how to tune the parameters of the model to make more accurate predictions.

## Pandas - Extracting data

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import mode

df = pd.read_csv('../data/train.csv')

## Pandas - Cleaning data

In [2]:
df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

age_mean = df['Age'].mean()
df['Age'] = df['Age'].fillna(age_mean)

#can not get to work
#mode_embarked = mode(df['Embarked'])[0][0]
#df['Embarked'] = df['Embarked'].fillna(mode_embarked)

# Replace missing values with most common port
df['Embarked'].isnull= df['Embarked'].dropna().mode().values

df['Gender'] = df['Sex'].map({'female': 0, 'male': 1}).astype(int)

pd.get_dummies(df['Embarked'], prefix='Embarked').head(10)
df = pd.concat([df, pd.get_dummies(df['Embarked'], prefix='Embarked')], axis=1)

df = df.drop(['Sex', 'Embarked'], axis=1)

cols = df.columns.tolist()
cols = [cols[1]] + cols[0:1] + cols[2:]

df = df[cols]

train_data = df.values

In [3]:
df.head(3)

,Survived,PassengerId,Pclass,Age,SibSp,Parch,Fare,Gender,Embarked_C,Embarked_Q,Embarked_S
0,0,1,3,22,1,0,7.2500,1,0,0,1
1,1,2,1,38,1,0,71.2833,0,1,0,0
2,1,3,3,26,0,0,7.9250,0,0,0,1


## Scikit-learn - Training the model

The documentation for the Random Forest Classifier details the different input parameters of the model. These input parameters include the number of trees, and the number of branches each tree has. It is unclear, off-the-bat, which values would be optimal. 

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

GridSearchCV allows us to test the desired range of input parameters, and review the performance of each set of values on a cross-validation basis. Here we review the number of features considered at each step a branch is made (max_features: 50% or 100% of features) and the maximum number of branches (max_depth: 5 levels or no limitations). 

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV

parameter_grid = {
    'max_features': [0.5, 1.],
    'max_depth': [10., None]
}

grid_search = GridSearchCV(RandomForestClassifier(n_estimators = 100), parameter_grid,
                            cv=5, verbose=3)

In [5]:
grid_search.fit(train_data[0:,2:], train_data[0:,0])

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] max_depth=10.0, max_features=0.5 ................................
[CV] ....... max_depth=10.0, max_features=0.5, score=0.804469 -   0.0s
[CV] max_depth=10.0, max_features=0.5 ................................
[CV] ....... max_depth=10.0, max_features=0.5, score=0.810056 -   0.0s
[CV] max_depth=10.0, max_features=0.5 ................................
[CV] ....... max_depth=10.0, max_features=0.5, score=0.882022 -   0.1s
[CV] max_depth=10.0, max_features=0.5 ................................
[CV] ....... max_depth=10.0, max_features=0.5, score=0.803371 -   0.0s
[CV] max_depth=10.0, max_features=0.5 ................................
[CV] ....... max_depth=10.0, max_features=0.5, score=0.864407 -   0.0s
[CV] max_depth=10.0, max_features=1.0 ................................
[CV] ....... max_depth=10.0, max_features=1.0, score=0.798883 -   0.0s
[CV] max_depth=10.0, max_features=1.0 ................................
[CV] ....... max_

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    2.8s finished


GridSearchCV(cv=5,
       estimator=RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
            min_samples_split=2, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'max_depth': [10.0, None], 'max_features': [0.5, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, score_func=None, scoring=None,
       verbose=3)

We now review the results.

In [6]:
grid_search.grid_scores_

[mean: 0.83277, std: 0.03349, params: {'max_depth': 10.0, 'max_features': 0.5},
 mean: 0.83053, std: 0.02306, params: {'max_depth': 10.0, 'max_features': 1.0},
 mean: 0.81706, std: 0.03525, params: {'max_depth': None, 'max_features': 0.5},
 mean: 0.81145, std: 0.03474, params: {'max_depth': None, 'max_features': 1.0}]

We sort the results, and determine the best-performing tuning parameters.

In [7]:
sorted(grid_search.grid_scores_, key=lambda x: x.mean_validation_score)
grid_search.best_score_
grid_search.best_params_

{'max_depth': 10.0, 'max_features': 0.5}

We then set these tuning parameters to our model.

In [8]:
model = RandomForestClassifier(n_estimators = 100, max_features=0.5, max_depth=10.0)
model = model.fit(train_data[0:,2:],train_data[0:,0])

**Exercise**

- Write the code so that grid_search refits model with the best tuning parameters to the entire dataset after these parameters are found, and hence allow us to skip the two lines of code above.

## Scikit-learn - Making predictions

In [9]:
df_test = pd.read_csv('../data/test.csv')

df_test = df_test.drop(['Name', 'Ticket', 'Cabin'], axis=1)

df_test['Age'] = df_test['Age'].fillna(age_mean)

fare_means = df.pivot_table('Fare', index='Pclass', aggfunc='mean')
df_test['Fare'] = df_test[['Fare', 'Pclass']].apply(lambda x:
                            fare_means[x['Pclass']] if pd.isnull(x['Fare'])
                            else x['Fare'], axis=1)

df_test['Gender'] = df_test['Sex'].map({'female': 0, 'male': 1}).astype(int)
df_test = pd.concat([df_test, pd.get_dummies(df_test['Embarked'], prefix='Embarked')],
                axis=1)

df_test = df_test.drop(['Sex', 'Embarked'], axis=1)

test_data = df_test.values

output = model.predict(test_data[:,1:])

C:\Users\Ethan\Anaconda3\lib\site-packages\pandas\core\index.py:648: FutureWarning: scalar indexers for index type Int64Index should be integers and not floating point
  type(self).__name__),FutureWarning)


In [10]:
fare_means

df_test.head(5)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Gender,Embarked_C,Embarked_Q,Embarked_S
0,892,3,34.5,0,0,7.8292,1,0,1,0
1,893,3,47.0,1,0,7.0000,0,0,0,1
2,894,2,62.0,0,0,9.6875,1,0,1,0
3,895,3,27.0,0,0,8.6625,1,0,0,1
4,896,3,22.0,1,1,12.2875,0,0,0,1


## Pandas - Preparing for submission

In [11]:
result = np.c_[test_data[:,0].astype(int), output.astype(int)]


df_result = pd.DataFrame(result[:,0:2], columns=['PassengerId', 'Survived'])
df_result.to_csv('../results/titanic_1-3.csv', index=False)